In [25]:
!pip install yfinance

# Bitcoin Price Prediction by using Cryptocurrency data from Yahoo finance API with the help of supervised learning- SVR (Support Vector Regression), Random Forest Regressor (RF), K-nearest neighgbour (KNN) and Deep learning - Long Short term Memory(LSTM). 

In [26]:
# Importing dataframe, math and Datetime libraries.
import pandas as pd
import numpy as np 
import math
import datetime as dt

# Machine learning libraries
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

#Deep leaening libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle

#Generating interactive graphs
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# Yahoo finance data connector for fetching the data.
import yfinance as yf

# Columns:

   #####  Date:      Date of Crypto transaction BTC-USD recorded by the exchange.
   #####  Open:      Opening price of the BTC-USD recorded by the exchange.
   #####  High:      Highest Price of BTC-USD recorded by the exchange.
   #####  Low:       Lowest Price of BTC-USD recorded by the exchange. 
   #####  Close:     Closing Price of BTC-USD recorded by the exchange.
   #####  Adj Close: Adjusted Closing Price of BTC-USD recorded by the exchange. 
   

In [27]:
start = dt.date(2020,1,1)
end = dt.date(2022,1,1)
ticker = 'BTC-USD'
df = yf.download(ticker,start,end)
df.head(6)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-31,7294.438965,7335.290039,7169.777832,7193.599121,7193.599121,21167946112
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095


In [28]:
df.insert(0,'Date',df.index)
df.reset_index(drop= True, inplace= True)

In [29]:
df.head(6)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-12-31,7294.438965,7335.290039,7169.777832,7193.599121,7193.599121,21167946112
1,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997
2,2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
3,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032
4,2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275
5,2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095


In [30]:
# Rename columns
df.rename(columns={"Date":"date","Open":"open","High":"high","Low":"low","Close":"close"}, inplace= True)
df.head()

,date,open,high,low,close,Adj Close,Volume
0,2019-12-31,7294.438965,7335.290039,7169.777832,7193.599121,7193.599121,21167946112
1,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997
2,2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
3,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032
4,2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275


### Insert shape here 

# Preprocessing Data



In [31]:
# Checking null value
df.isnull().sum()

date         0
open         0
high         0
low          0
close        0
Adj Close    0
Volume       0
dtype: int64

In [32]:
df.describe()

,open,high,low,close,Adj Close,Volume
count,732.000000,732.000000,732.000000,732.000000,732.000000,7.320000e+02
mean,29174.659524,29977.083749,28295.454552,29221.677826,29221.677826,4.005401e+10
std,19696.722229,20230.628222,19063.461255,19682.977362,19682.977362,2.057345e+10
min,5002.578125,5331.833984,4106.980957,4970.788086,4970.788086,1.225260e+10
25%,9690.349609,9836.288818,9539.769775,9697.846191,9697.846191,2.747029e+10
50%,28100.832031,29091.308594,27781.041016,28921.336914,28921.336914,3.615254e+10
75%,47780.924805,49161.133789,46666.096680,47729.456055,47729.456055,4.733599e+10
max,67549.734375,68789.625000,66382.062500,67566.828125,67566.828125,3.509679e+11


In [33]:
# Checking na value
df.isna().any()

date         False
open         False
high         False
low          False
close        False
Adj Close    False
Volume       False
dtype: bool

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       732 non-null    datetime64[ns]
 1   open       732 non-null    float64       
 2   high       732 non-null    float64       
 3   low        732 non-null    float64       
 4   close      732 non-null    float64       
 5   Adj Close  732 non-null    float64       
 6   Volume     732 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 40.2 KB


In [35]:
# Sorting dataset by date format

In [36]:
df.sort_values(by='date', inplace=True)
df.head(6)

,date,open,high,low,close,Adj Close,Volume
0,2019-12-31,7294.438965,7335.290039,7169.777832,7193.599121,7193.599121,21167946112
1,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997
2,2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
3,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032
4,2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275
5,2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095


In [37]:
print("The shape and size of the data is",format(df.shape))

The shape and size of the data is (732, 7)


# EDA - Exploratory Data Analysis



In [38]:
#print("Starting date: ",df.iloc[0][0])
print("Ending date: ", df.iloc[-1][0])
print("Duration: ", df.iloc[-1][0]-df.iloc[0][0])

Ending date:  2021-12-31 00:00:00
Duration:  731 days 00:00:00


# Monthwise comparision between Stock actual, open and close price


In [39]:
monthvise= df.groupby(df['date'].dt.strftime('%B,%Y'))[['open','close']].mean().sort_values(by='close')
monthvise.head(6)

,open,close
date,,
"March,2020",6943.507009,6871.016113
"December,2019",7294.438965,7193.599121
"April,2020",7150.611328,7224.477327
"January,2020",8318.949597,8389.270476
"May,2020",9237.761530,9263.151745
"June,2020",9499.797005,9489.227214


pip instal performance analysis package

In [40]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['open'],
    name='Stock Open Price',
    marker_color='teal'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock actual, open and close price')
fig.show()

# Monthwise High and Low stock price

In [41]:
df.groupby(df['date'].dt.strftime('%B,%Y'))['low'].min()

date
April,2020         6202.373535
April,2021        47159.484375
August,2020       11012.415039
August,2021       37458.003906
December,2019      7169.777832
December,2020     17619.533203
December,2021     42874.617188
February,2020      8492.932617
February,2021     32384.228516
January,2020       6914.996094
January,2021      28722.755859
July,2020          8977.015625
July,2021         29360.955078
June,2020          8975.525391
June,2021         28893.621094
March,2020         4106.980957
March,2021        45115.093750
May,2020           8374.323242
May,2021          30681.496094
November,2020     13243.160156
November,2021     53569.765625
October,2020      10416.689453
October,2021      43320.023438
September,2020     9916.493164
September,2021    39787.609375
Name: low, dtype: float64

In [42]:
monthvise_high= df.groupby(df['date'].dt.strftime('%B,%Y'))['high'].max()
monthvise_low= df.groupby(df['date'].dt.strftime('%B,%Y'))['low'].min()

In [43]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='teal'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='violet'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title=' Monthwise High and Low stock price')
fig.show()

# Trend comparision between stock price, open price, close price, high price, low price


trend analysis, research on web, 

In [44]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(df, x=df.date, y=[df['open'], df['close'], df['high'], df['low']], labels={'date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# Close price prediction preparation and preprocessing

## Making separate dataframe with close price

In [45]:
closedf = df[['date','close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (732, 2)


In [46]:
closedf

,date,close
0,2019-12-31,7193.599121
1,2020-01-01,7200.174316
2,2020-01-02,6985.470215
3,2020-01-03,7344.884277
4,2020-01-04,7410.656738
...,...,...
727,2021-12-27,50640.417969
728,2021-12-28,47588.855469
729,2021-12-29,46444.710938
730,2021-12-30,47178.125000


## Plotting stock close price chart


In [47]:
fig = px.line(closedf, x=closedf.date, y=closedf.close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.7)
fig.update_layout(title_text='Stock close price chart', plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

## Normalizing / scaling close value between 0 to 1


In [48]:
close_stock = closedf.copy()
del closedf['date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(732, 1)


In [49]:
closedf


array([[3.55104098e-02],
       [3.56154515e-02],
       [3.21854566e-02],
       [3.79272585e-02],
       [3.89780033e-02],
       [3.89885573e-02],
       [4.47062011e-02],
       [5.10080877e-02],
       [4.96688721e-02],
       [4.64611372e-02],
       [5.10538065e-02],
       [4.89927080e-02],
       [5.14682087e-02],
       [5.06965975e-02],
       [6.16169419e-02],
       [6.12853889e-02],
       [5.99558382e-02],
       [6.32348308e-02],
       [6.34548209e-02],
       [5.96756125e-02],
       [5.88991650e-02],
       [6.03090298e-02],
       [5.92703291e-02],
       [5.48872986e-02],
       [5.55090463e-02],
       [5.42695603e-02],
       [5.79276579e-02],
       [6.29278026e-02],
       [7.00971141e-02],
       [6.94267847e-02],
       [7.24998750e-02],
       [6.99683432e-02],
       [7.06448349e-02],
       [6.98698695e-02],
       [6.90576176e-02],
       [6.72594433e-02],
       [7.41682020e-02],
       [7.60273920e-02],
       [7.70840339e-02],
       [7.81891483e-02],


## Split data for training and testing


In [50]:
training_size=int(len(closedf)*0.80)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (585, 1)
test_data:  (147, 1)


# Create new dataset according to requirement of time-series prediction


In [51]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [52]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (569, 15)
y_train:  (569,)
X_test:  (131, 15)
y_test (131,)


# *Algorithms*



## *Support vector regression - SVR*


In [53]:
from sklearn.svm import SVR

svr_rbf = SVR(kernel= 'rbf', C= 1e2, gamma= 'scale')
svr_rbf.fit(X_train, y_train)

SVR(C=100.0)

In [102]:
#Specific price prediction is gotten from here

start = dt.date(2022,6,1)
end = dt.date(2022,6,15)
ticker = 'BTC-USD'
df_pred = yf.download(ticker,start,end)
df_pred.head

closed_pred = df_pred[['Close']]

closed_pred = closed_pred.values
closed_pred = closed_pred.reshape((1 , 15))

close_stock = closed_pred.copy()
scaler=MinMaxScaler(feature_range=(0,1))
closed_=scaler.fit_transform(close_stock)

answer = svr_rbf.predict(closed_)
trainPredict_dataset_like = np.zeros(shape=(1, 15))
trainPredict_dataset_like[:,0] = answer
trainPredict = scaler.inverse_transform(trainPredict_dataset_like)[:,0]
trainPredict

[*********************100%***********************]  1 of 1 completed


array([31792.4212928])

In [54]:
train_predict=svr_rbf.predict(X_train)
test_predict=svr_rbf.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)

Train data prediction: (569, 1)
Test data prediction: (131, 1)


In [55]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

### Evaluation metrices RMSE, MSE and MAE
#### Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data

In [56]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  2622.88867211713
Train data MSE:  6879544.986320363
Test data MAE:  2176.4879166204287
-------------------------------------------------------------------------------------
Test data RMSE:  3646.676554267739
Test data MSE:  13298249.89144603
Test data MAE:  2976.1219207281174


### Explained variance regression score
#### The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [57]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9815246095899588
Test data explained variance regression score: 0.8288590430274618


feature reduction by adding and removing column

### R2 score for regression
#### R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best
0 or < 0 = worse

In [58]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9774623738585543
Test data R2 score: 0.7226792289619226


#### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)


In [59]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.03647193976116737
Test data MGD:  0.004434656407278192
----------------------------------------------------------------------
Train data MPD:  421.0934406399879
Test data MPD:  241.35346333755442


#### Comparision between original stock close price vs predicted close price

In [60]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close_svr': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close_svr': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close_svr'],
                                          plotdf['test_predicted_close_svr']],
              labels={'value':'Bitcoin price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (732, 1)
Test predicted data:  (732, 1)


## Random Forest Regressor - RF

In [61]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 1200, random_state = 0)
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1200, random_state=0)

In [62]:
# The prediction 

train_predict=regressor.predict(X_train)
test_predict=regressor.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)

Train data prediction: (569, 1)
Test data prediction: (131, 1)


In [101]:
#Specific price prediction is gotten from here

start = dt.date(2022,6,1)
end = dt.date(2022,6,15)
ticker = 'BTC-USD'
#df_pred = yf.download(ticker,start,end)
#df_pred.head

closed_pred = df_pred[['Close']]

closed_pred = closed_pred.values
closed_pred = closed_pred.reshape((1 , 15))

close_stock = closed_pred.copy()
scaler=MinMaxScaler(feature_range=(0,1))
closed_=scaler.fit_transform(close_stock)

answer = regressor.predict(closed_)
trainPredict_dataset_like = np.zeros(shape=(1, 15))
trainPredict_dataset_like[:,0] = answer
trainPredict = scaler.inverse_transform(trainPredict_dataset_like)[:,0]
trainPredict


array([31792.32944691])

In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

## Evaluation metrices RMSE, MSE and MAE
### Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

## Explained variance regression score
### The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

## R2 score for regression
### R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best
0 or < 0 = worse

In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

## Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

## Comparision between original stock close price vs predicted close price


In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close_rfr': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close_rfr': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close_rfr'],
                                          plotdf['test_predicted_close_rfr']],
              labels={'value':'Bitcoin price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

## K-nearest neighgbour - KNN

In [104]:
from sklearn import neighbors

K = time_step
neighbor = neighbors.KNeighborsRegressor(n_neighbors = K)
neighbor.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=15)

In [105]:
# The prediction 

train_predict=neighbor.predict(X_train)
test_predict=neighbor.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)

Train data prediction: (569, 1)
Test data prediction: (131, 1)


In [106]:
#Specific price prediction is gotten from here

start = dt.date(2022,6,1)
end = dt.date(2022,6,15)
ticker = 'BTC-USD'
df_pred = yf.download(ticker,start,end)
df_pred.head

closed_pred = df_pred[['Close']]

closed_pred = closed_pred.values
closed_pred = closed_pred.reshape((1 , 15))

close_stock = closed_pred.copy()
scaler=MinMaxScaler(feature_range=(0,1))
closed_=scaler.fit_transform(close_stock)

answer = neighbor.predict(closed_)
trainPredict_dataset_like = np.zeros(shape=(1, 15))
trainPredict_dataset_like[:,0] = answer
trainPredict = scaler.inverse_transform(trainPredict_dataset_like)[:,0]
trainPredict

array([31792.33750667])

In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

## Evaluation metrices RMSE, MSE and MAE
### Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

## Explained variance regression score
### The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

## R2 score for regression
### R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best
0 or < 0 = worse



In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

## Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

## Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close_knn': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close_knn': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close_knn'],
                                          plotdf['test_predicted_close_knn']],
              labels={'value':'Bitcoin price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

## LSTM (Long Short Term Memory)

In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

## LSTM model structure

In [107]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

2022-06-15 14:17:49.845879: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [108]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 15, 32)            8320      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 21,025
Trainable params: 21,025
Non-trainable params: 0
_________________________________________________________________


In [109]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30,batch_size=2,verbose=1)

Epoch 1/30
285/285 [==============================] - 9s 17ms/step - loss: 0.0078 - val_loss: 0.0060
Epoch 2/30
285/285 [==============================] - 3s 11ms/step - loss: 0.0043 - val_loss: 0.0045
Epoch 3/30
285/285 [==============================] - 3s 11ms/step - loss: 0.0034 - val_loss: 0.0172
Epoch 4/30
285/285 [==============================] - 4s 16ms/step - loss: 0.0028 - val_loss: 0.0034
Epoch 5/30
285/285 [==============================] - 3s 12ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 6/30
285/285 [==============================] - 4s 13ms/step - loss: 0.0017 - val_loss: 0.0041
Epoch 7/30
285/285 [==============================] - 3s 12ms/step - loss: 0.0018 - val_loss: 0.0015
Epoch 8/30
285/285 [==============================] - 3s 12ms/step - loss: 0.0013 - val_loss: 0.0025
Epoch 9/30
285/285 [==============================] - 3s 12ms/step - loss: 0.0011 - val_loss: 0.0020
Epoch 10/30
285/285 [==============================] - 3s 11ms/step - loss: 0.0010 - val_lo

In [110]:
#Specific price prediction is gotten from here

start = dt.date(2022,6,1)
end = dt.date(2022,6,15)
ticker = 'BTC-USD'
#df_pred = yf.download(ticker,start,end)
#df_pred.head

closed_pred = df_pred[['Close']]

closed_pred = closed_pred.values
closed_pred = closed_pred.reshape((1 , 15))

close_stock = closed_pred.copy()
scaler=MinMaxScaler(feature_range=(0,1))
closed_=scaler.fit_transform(close_stock)

answer = model.predict(closed_)
trainPredict_dataset_like = np.zeros(shape=(1, 15))
trainPredict_dataset_like[:,0] = answer
trainPredict = scaler.inverse_transform(trainPredict_dataset_like)[:,0]
trainPredict

array([31792.31856058])

In [ ]:
### The prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

In [ ]:
# Transform back to original form.

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

## Evaluation metrices RMSE, MSE and MAE
### Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

## Explained variance regression score¶
### The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

## R2 score for regression
### R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best
0 or < 0 = worse

In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

## Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)


In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

## Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close_lstm': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close_lstm': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close_lstm'],
                                          plotdf['test_predicted_close_lstm']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()